## Combine GOES netCDFs into one .zarr file

This could also be done using the `goes-ortho.build_zarr()` function but that is currently not working.

Clinton Alden
27 March 2025

In [1]:
import xarray as xr
import os
import dask
from dask.distributed import Client

In [2]:
def goes_nc_to_zarr(in_dir, out_dir, out_name):    
# Directory containing the NetCDF files
    nc_dir = in_dir
    
    # Recursively list all NetCDF files in the directory and subdirectories
    nc_files = []
    for root, dirs, files in os.walk(nc_dir):
        for file in files:
            print(file)
            if file.endswith('.nc'):
                nc_files.append(os.path.join(root, file))

    # Open multiple NetCDF files as a list of datasets
    datasets = [xr.open_dataset(f) for f in nc_files]

    # Concatenate datasets along the 't' coordinate
    combined_ds = xr.concat(datasets, dim='t')

    # Save the combined dataset to a Zarr file
    out_name = out_name
    combined_ds.to_zarr(out_dir+out_name)

In [3]:
for day in range(2,21):
    day_of_month = str(day)
    if day<10:
        out_day_of_month = '0' + day_of_month
    else:
        out_day_of_month = day_of_month
    print(f'Starting June {day_of_month}')
    in_dir = f'/storage/cdalden/goes/goes16/2023/6/{day_of_month}/'
    out_dir = '/storage/cdalden/goes/goes16/2023/'
    out_name = f'goes16_C02_colorado_202306{out_day_of_month}.zarr'

    goes_nc_to_zarr(in_dir, out_dir, out_name)
    print(f'Finished June {day_of_month}')

Starting June 2
OR_ABI-L1b-RadC-M6C02_G16_s20231530001175_e20231530003548_c20231530003585.nc
OR_ABI-L1b-RadC-M6C02_G16_s20231530006175_e20231530008548_c20231530008583.nc
OR_ABI-L1b-RadC-M6C02_G16_s20231530011175_e20231530013548_c20231530013584.nc
OR_ABI-L1b-RadC-M6C02_G16_s20231530016175_e20231530018548_c20231530018588.nc
OR_ABI-L1b-RadC-M6C02_G16_s20231530021175_e20231530023548_c20231530023580.nc
OR_ABI-L1b-RadC-M6C02_G16_s20231530026175_e20231530028548_c20231530028581.nc
OR_ABI-L1b-RadC-M6C02_G16_s20231530031175_e20231530033548_c20231530033588.nc
OR_ABI-L1b-RadC-M6C02_G16_s20231530036175_e20231530038548_c20231530038578.nc
OR_ABI-L1b-RadC-M6C02_G16_s20231530041175_e20231530043548_c20231530043582.nc
OR_ABI-L1b-RadC-M6C02_G16_s20231530046175_e20231530048548_c20231530048579.nc
OR_ABI-L1b-RadC-M6C02_G16_s20231530051175_e20231530053548_c20231530053582.nc
OR_ABI-L1b-RadC-M6C02_G16_s20231530056175_e20231530058548_c20231530058584.nc
OR_ABI-L1b-RadC-M6C02_G16_s20231530101175_e20231530103548_c2

/home/cdalden/miniforge3/envs/goesenv/lib/python3.13/site-packages/zarr/api/asynchronous.py:203: UserWarning: Consolidated metadata is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  warnings.warn(


Finished June 2
Starting June 3
OR_ABI-L1b-RadC-M6C02_G16_s20231540001176_e20231540003549_c20231540003584.nc
OR_ABI-L1b-RadC-M6C02_G16_s20231540006176_e20231540008549_c20231540008590.nc
OR_ABI-L1b-RadC-M6C02_G16_s20231540011176_e20231540013549_c20231540013586.nc
OR_ABI-L1b-RadC-M6C02_G16_s20231540016176_e20231540018549_c20231540018587.nc
OR_ABI-L1b-RadC-M6C02_G16_s20231540021176_e20231540023549_c20231540023587.nc
OR_ABI-L1b-RadC-M6C02_G16_s20231540026176_e20231540028549_c20231540028578.nc
OR_ABI-L1b-RadC-M6C02_G16_s20231540031176_e20231540033549_c20231540033584.nc
OR_ABI-L1b-RadC-M6C02_G16_s20231540036176_e20231540038549_c20231540038576.nc
OR_ABI-L1b-RadC-M6C02_G16_s20231540041176_e20231540043549_c20231540043580.nc
OR_ABI-L1b-RadC-M6C02_G16_s20231540046176_e20231540048549_c20231540048582.nc
OR_ABI-L1b-RadC-M6C02_G16_s20231540051176_e20231540053549_c20231540053577.nc
OR_ABI-L1b-RadC-M6C02_G16_s20231540056176_e20231540058549_c20231540058584.nc
OR_ABI-L1b-RadC-M6C02_G16_s20231540101176_e2

/home/cdalden/miniforge3/envs/goesenv/lib/python3.13/site-packages/zarr/api/asynchronous.py:203: UserWarning: Consolidated metadata is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  warnings.warn(


Finished June 3
Starting June 4
OR_ABI-L1b-RadC-M6C02_G16_s20231550001177_e20231550003550_c20231550003588.nc
OR_ABI-L1b-RadC-M6C02_G16_s20231550006177_e20231550008550_c20231550008584.nc
OR_ABI-L1b-RadC-M6C02_G16_s20231550011177_e20231550013550_c20231550013587.nc
OR_ABI-L1b-RadC-M6C02_G16_s20231550016177_e20231550018550_c20231550018585.nc
OR_ABI-L1b-RadC-M6C02_G16_s20231550021177_e20231550023550_c20231550023589.nc
OR_ABI-L1b-RadC-M6C02_G16_s20231550026177_e20231550028550_c20231550028588.nc
OR_ABI-L1b-RadC-M6C02_G16_s20231550031177_e20231550033550_c20231550033585.nc
OR_ABI-L1b-RadC-M6C02_G16_s20231550036177_e20231550038550_c20231550038584.nc
OR_ABI-L1b-RadC-M6C02_G16_s20231550041177_e20231550043550_c20231550043588.nc
OR_ABI-L1b-RadC-M6C02_G16_s20231550046177_e20231550048550_c20231550048585.nc
OR_ABI-L1b-RadC-M6C02_G16_s20231550051177_e20231550053550_c20231550053580.nc
OR_ABI-L1b-RadC-M6C02_G16_s20231550056177_e20231550058550_c20231550058585.nc
OR_ABI-L1b-RadC-M6C02_G16_s20231550101177_e2

...

Finished June 18
Starting June 19
OR_ABI-L1b-RadC-M6C02_G16_s20231700001181_e20231700003555_c20231700003590.nc
OR_ABI-L1b-RadC-M6C02_G16_s20231700006181_e20231700008554_c20231700008586.nc
OR_ABI-L1b-RadC-M6C02_G16_s20231700011181_e20231700013554_c20231700013593.nc
OR_ABI-L1b-RadC-M6C02_G16_s20231700016181_e20231700018554_c20231700018598.nc
OR_ABI-L1b-RadC-M6C02_G16_s20231700021181_e20231700023554_c20231700023596.nc
OR_ABI-L1b-RadC-M6C02_G16_s20231700026181_e20231700028554_c20231700028590.nc
OR_ABI-L1b-RadC-M6C02_G16_s20231700031181_e20231700033554_c20231700033589.nc
OR_ABI-L1b-RadC-M6C02_G16_s20231700036181_e20231700038554_c20231700038591.nc
OR_ABI-L1b-RadC-M6C02_G16_s20231700041181_e20231700043554_c20231700043587.nc
OR_ABI-L1b-RadC-M6C02_G16_s20231700046181_e20231700048554_c20231700048586.nc
OR_ABI-L1b-RadC-M6C02_G16_s20231700051181_e20231700053554_c20231700053590.nc
OR_ABI-L1b-RadC-M6C02_G16_s20231700056181_e20231700058554_c20231700058588.nc
OR_ABI-L1b-RadC-M6C02_G16_s20231700101181_

/home/cdalden/miniforge3/envs/goesenv/lib/python3.13/site-packages/zarr/api/asynchronous.py:203: UserWarning: Consolidated metadata is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  warnings.warn(


Finished June 19
Starting June 20
OR_ABI-L1b-RadC-M6C02_G16_s20231710001183_e20231710003556_c20231710003590.nc
OR_ABI-L1b-RadC-M6C02_G16_s20231710006183_e20231710008556_c20231710008589.nc
OR_ABI-L1b-RadC-M6C02_G16_s20231710011183_e20231710013556_c20231710013589.nc
OR_ABI-L1b-RadC-M6C02_G16_s20231710016183_e20231710018556_c20231710018586.nc
OR_ABI-L1b-RadC-M6C02_G16_s20231710021183_e20231710023556_c20231710023596.nc
OR_ABI-L1b-RadC-M6C02_G16_s20231710026183_e20231710028556_c20231710029008.nc
OR_ABI-L1b-RadC-M6C02_G16_s20231710031183_e20231710033556_c20231710033591.nc
OR_ABI-L1b-RadC-M6C02_G16_s20231710036183_e20231710038556_c20231710038594.nc
OR_ABI-L1b-RadC-M6C02_G16_s20231710041183_e20231710043556_c20231710043591.nc
OR_ABI-L1b-RadC-M6C02_G16_s20231710046183_e20231710048556_c20231710048592.nc
OR_ABI-L1b-RadC-M6C02_G16_s20231710051183_e20231710053556_c20231710053592.nc
OR_ABI-L1b-RadC-M6C02_G16_s20231710056183_e20231710058556_c20231710058588.nc
OR_ABI-L1b-RadC-M6C02_G16_s20231710101183_

/home/cdalden/miniforge3/envs/goesenv/lib/python3.13/site-packages/zarr/api/asynchronous.py:203: UserWarning: Consolidated metadata is currently not part in the Zarr format 3 specification. It may not be supported by other zarr implementations and may change in the future.
  warnings.warn(


Finished June 20


## Combine all zarr files for each date into one file

In [4]:
def combine_zarr_files(input_dir, output_file):
    # List all Zarr files in the input directory
    zarr_files = [os.path.join(input_dir, f) for f in os.listdir(input_dir) if f.endswith('.zarr')]

    # Open each Zarr file as an xarray.Dataset
    datasets = [xr.open_zarr(zarr_file) for zarr_file in zarr_files]

    # Concatenate datasets along the 't' dimension
    combined_ds = xr.concat(datasets, dim='t')

    # Save the combined dataset to a new Zarr file
    combined_ds.to_zarr(output_file, mode='w')

# Example usage
input_dir = '/storage/cdalden/goes/goes16/2023/'
output_file = '/storage/cdalden/goes/goes16/2023/goes16_C02_colorado_202306_combined.zarr'

combine_zarr_files(input_dir, output_file)

/home/cdalden/miniforge3/envs/goesenv/lib/python3.13/site-packages/dask/array/core.py:5097: PerformanceWarning: Increasing number of chunks by factor of 27
  result = blockwise(
/home/cdalden/miniforge3/envs/goesenv/lib/python3.13/site-packages/dask/array/core.py:5097: PerformanceWarning: Increasing number of chunks by factor of 27
  result = blockwise(
/home/cdalden/miniforge3/envs/goesenv/lib/python3.13/site-packages/dask/array/core.py:5097: PerformanceWarning: Increasing number of chunks by factor of 46
  result = blockwise(
/home/cdalden/miniforge3/envs/goesenv/lib/python3.13/site-packages/dask/array/core.py:5097: PerformanceWarning: Increasing number of chunks by factor of 46
  result = blockwise(
/home/cdalden/miniforge3/envs/goesenv/lib/python3.13/site-packages/dask/array/core.py:5097: PerformanceWarning: Increasing number of chunks by factor of 46
  result = blockwise(
/home/cdalden/miniforge3/envs/goesenv/lib/python3.13/site-packages/dask/array/core.py:5097: PerformanceWarning

ValueError: Specified zarr chunks encoding['chunks']=(279,) for variable named 'nominal_satellite_subpoint_lon' would overlap multiple dask chunks ((279, 288, 288, 288, 288, 288, 288, 288, 288, 287, 288, 288, 288, 288, 288, 288, 288, 288, 288, 286),) on the region (slice(None, None, None),). Writing this array in parallel with dask could lead to corrupted data. Consider either rechunking using `chunk()`, deleting or modifying `encoding['chunks']`, or specify `safe_chunks=False`.